In [ ]:
#| label: qmtFig6jn

import scipy
import scipy.io as sio
from pathlib import Path
import numpy as np

from contextlib import contextmanager
import sys, os
from pathlib import Path

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

import os
from pathlib import Path

def find_myst_yml_directories(start_dir=None):
    """
    Recursively search for directories containing myst.yml file.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        list: List of full paths to directories containing myst.yml
    """
    if start_dir is None:
        start_dir = Path.cwd()
    else:
        start_dir = Path(start_dir)
    
    myst_dirs = []
    
    def _search_directory(current_dir):
        # Check if myst.yml exists in current directory
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            myst_dirs.append(str(current_dir.resolve()))
            # Don't search subdirectories if we found myst.yml here
            return
        
        # Recursively search all subdirectories
        for item in current_dir.iterdir():
            if item.is_dir():
                try:
                    _search_directory(item)
                except (PermissionError, OSError):
                    # Skip directories we can't access
                    continue
    
    _search_directory(start_dir)
    return myst_dirs

def find_myst_yml_directories_upwards(start_dir=None):
    """
    Search for myst.yml in current directory, if not found go to parent and repeat.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        str or None: Full path of directory containing myst.yml, or None if not found
    """
    if start_dir is None:
        current_dir = Path.cwd()
    else:
        current_dir = Path(start_dir)
    
    # Keep going up until we reach the filesystem root
    while current_dir != current_dir.parent:  # Stop at root
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            return str(current_dir.resolve())
        
        # Move to parent directory
        current_dir = current_dir.parent
    
    return None

with suppress_stdout():
    repo_path = Path(find_myst_yml_directories_upwards())
    print(repo_path)
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
data_dir = dataset_path / "06-MT-01-qMT" / "06-MT-01-qMT" 

b0map_mat = scipy.io.loadmat(data_dir / 'qMT_tutorial-ISMRM2022-main' / 'results/realData/b0mapMAT.mat')
b0map = np.array(b0map_mat["b0mapMAT"])
b1map_mat = scipy.io.loadmat(data_dir / 'qMT_tutorial-ISMRM2022-main' / 'results/realData/b1mapMAT.mat')
b1map = np.array(b1map_mat["b1mapMAT"])
r1map_mat = scipy.io.loadmat(data_dir / 'qMT_tutorial-ISMRM2022-main' / 'results/realData/r1mapMAT.mat')
r1map = np.array(r1map_mat["r1mapMAT"])
mtdata_mat = scipy.io.loadmat(data_dir / 'qMT_tutorial-ISMRM2022-main' / 'results/realData/mtdataMAT.mat')
mtdata = np.array(mtdata_mat["mtdataMAT"])
mask_mat = scipy.io.loadmat(data_dir / 'qMT_tutorial-ISMRM2022-main' / 'results/realData/maskMAT.mat')
mask = np.array(mask_mat["maskMAT"])

Fmap_mat = scipy.io.loadmat(data_dir / 'qMT_tutorial-ISMRM2022-main' / 'results/realData/FmapMAT.mat')
Fmap = np.array(Fmap_mat["FmapMAT"])
kfmap_mat = scipy.io.loadmat(data_dir / 'qMT_tutorial-ISMRM2022-main' / 'results/realData/kfmapMAT.mat')
kfmap = np.array(kfmap_mat["kfmapMAT"])
T2fmap_mat = scipy.io.loadmat(data_dir / 'qMT_tutorial-ISMRM2022-main' / 'results/realData/T2fmapMAT.mat')
T2fmap = np.array(T2fmap_mat["T2f"])
T2rmap_mat = scipy.io.loadmat(data_dir / 'qMT_tutorial-ISMRM2022-main' / 'results/realData/T2rmapMAT.mat')
T2rmap = np.array(T2rmap_mat["T2r"])

import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.image import imread
import scipy.io
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

# PYTHON CODE
from plotly import tools

xAxis = np.arange(0,88)
yAxis = np.arange(0,128)

b0map = np.multiply(b0map,mask)
b1map = np.multiply(b1map,mask)
r1map = np.multiply(r1map,mask)
mtdata_angle = np.multiply(mtdata[:,:,:,0].reshape((88,128)),mask)
Fmap = np.multiply(Fmap,mask)
kfmap = np.multiply(kfmap,mask)
T2fmap = np.multiply(T2fmap,mask)
T2rmap = np.multiply(T2rmap,mask)

trace1 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(b0map,3),
                   zmin=-30,
                   zmax=30,
                   colorscale='Portland',
                   colorbar=dict(title = "B<sub>0</sub> (Hz)", thickness=35,
                           tickfont=dict(family='Times New Roman', size=15), ypad=1),
                   visible=True,
                   name = 'B<sub>0</sub> (Hz)')
trace2 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(b1map,3),
                   zmin=0.7,
                   zmax=1.3,
                   colorscale='Portland',
                   colorbar=dict(title = "B<sub>1</sub>", thickness=35,
                           tickfont=dict(family='Times New Roman', size=15), ypad=1),
                   visible=False,
                   name = 'B<sub>1</sub> values')
trace3 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(r1map,3),
                   zmin=0.5,
                   zmax=1.2,
                   colorscale='Portland',
                   colorbar=dict(title = "R<sub>1</sub> (s<sup>-1</sup>)", thickness=35,
                           tickfont=dict(family='Times New Roman', size=15), ypad=1),
                   visible=False,
                   name = 'R<sub>1</sub> (s<sup>-1</sup>)')
trace4 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(mtdata_angle,3),
                   zmin=0.5,
                   zmax=1,
                   colorscale='Portland',
                   colorbar=dict(title = "MT", thickness=35,
                           tickfont=dict(family='Times New Roman', size=15), ypad=1),
                   visible=False,
                   name = 'MT (angle 142)')
trace5 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(Fmap,3),
                   zmin=0.05,
                   zmax=0.3,
                   colorscale='Portland',
                   colorbar=dict(title = "F", thickness=35,
                           tickfont=dict(family='Times New Roman', size=15), ypad=1),
                   visible=True,
                   name = 'F')
trace6 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(kfmap,3),
                   colorscale='Portland',
                   colorbar=dict(title = "k<sub>f</sub> (s<sup>-1</sup>)", thickness=35,
                           tickfont=dict(family='Times New Roman', size=15), ypad=1),
                   visible=False,
                   name = 'k<sub>f</sub> (s<sup>-1</sup>)')
trace7 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(T2fmap,3),
                   zmin=0,
                   zmax=0.05,
                   colorscale='Portland',
                   colorbar=dict(title = "T<sub>2f</sub> (ms)", thickness=35,
                           tickfont=dict(family='Times New Roman', size=15)),
                   visible=False,
                   name = 'T<sub>2f</sub> (ms)')
trace8 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=np.rot90(T2rmap,3),
                   zmin=5e-6,
                   zmax=25e-6,
                   colorscale='Portland',
                   colorbar=dict(title = "T<sub>2r</sub> (s)", thickness=35, ticklen=7,
                           tickfont=dict(family='Times New Roman', size=15)),
                   visible=False,
                   name = 'T<sub>2r</sub> (μs)')

data1=[trace1, trace2, trace3, trace4]
data2=[trace5, trace6, trace7, trace8]

updatemenus = list([
    dict(active=0,
         x = 0.2,
         xanchor = 'left',
         y = -0.15,
         yanchor = 'bottom',
         direction = 'up',
         font=dict(
                family='Times New Roman',
                size=16
            ),
         buttons=list([   
            dict(label = 'B0 map',
                 method = 'update',
                 args = [{'visible': [True, False, False, False]},
                         ]),
            dict(label = 'B1 map',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                         ]),
            dict(label = 'R1 map',
                 method = 'update',
                 args = [{'visible': [False, False, True, False]},
                         ]),
            dict(label = 'MT data',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                         ])
        ]),
    )
])

layout1 = dict(
    width=345,
    height=345,
    margin = dict(
                t=40,
                r=50,
                b=10,
                l=50),
    annotations=[
        dict(
            x=0.5,
            y=1.15,
            showarrow=False,
            text='Input maps',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        )
    ],
    xaxis = dict(range = [0,85], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    yaxis = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    showlegend = False,
    autosize = False,
    updatemenus=updatemenus
)

fig = dict(data=data1,layout=layout1)

iplot(fig, filename = 'basic-heatmap', config = config)

In [ ]:
#| label: qmtFig7cell

updatemenus = list([    
    dict(active=0,
         x = 0.2,
         xanchor = 'left',
         y = -0.15,
         yanchor = 'bottom',
         direction = 'up',
         font=dict(
                family='Times New Roman',
                size=16),
         buttons=list([   
            dict(label = 'F map',
                 method = 'update',
                 args = [{'visible': [True, False, False, False]},
                         ]),
            dict(label = 'kf map',
                 method = 'update',
                 args = [{'visible': [False, True, False, False]},
                         ]),
            dict(label = 'T2f map',
                 method = 'update',
                 args = [{'visible': [False, False, True, False]},
                         ]),
             dict(label = 'T2r map',
                 method = 'update',
                 args = [{'visible': [False, False, False, True]},
                         ])
        ]),
    )
])

layout2 = dict(
    width=345,
    height=345,
    margin = dict(
                t=40,
                r=50,
                b=10,
                l=50),
    annotations=[
        dict(
            x=0.5,
            y=1.15,
            showarrow=False,
            text='Fitted maps',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        )
    ],
    xaxis = dict(range = [0,85], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    yaxis = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    showlegend = False,
    autosize = False,
    updatemenus=updatemenus
)

fig = dict(data=data2,layout=layout2)

iplot(fig, filename = 'basic-heatmap', config = config)